##### 작성자  : 손해원
- 주 기능 : 네이버 블로그에서 키워드로 검색 후 포스팅 원문을 수집합니다. 
- 수집 가능 항목 : 원문url, 글제목, 글쓴이, 작성날짜, 원문, 해쉬태그 

In [1]:
import pandas as pd
from selenium import webdriver
import time
from tqdm import tqdm 

In [2]:
driver = webdriver.Chrome('/home/haewon/chromedriver') #### 드라이버 경로 설정 

##### 검색어, 날짜 설정 
- 최근 1년동안의 url만 모으기 위해 시작 날짜와 마지막 날짜를 설정합니다.

In [ ]:
keyword = '부산%20맛집'  ### 검색할 키워드 설정 
startDate ='2020-11-01' ### 시작날짜
endDate = '2021-11-23'  ### 마지막날짜 

##### 드라이버 오픈

In [3]:
url = f'https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={keyword}'
driver.get(url)

##### url 수집

In [ ]:
search_number = int(driver.find_element_by_class_name('search_number').text.replace(',','').replace('건','')) # 총 게시물 수 
hrefs = driver.find_elements_by_class_name('desc_inner')
url_list = []
num = 1 

for num in tqdm(range(1, int(search_number/7))):  # 총 게시물 수/7(한 페이지당 게시물 수)
    for i in range(len(hrefs)):
        
        # url 수집 후 리스트에 저장 
        time.sleep(0.5)
        href = driver.find_elements_by_class_name('desc_inner')[i].get_attribute('href')
        url_list.append(href)
        
    # 다음 페이지로 이동 
    num += 1 
    driver.get(f'https://section.blog.naver.com/Search/Post.naver?pageNo={num}&rangeType=PERIOD&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={keyword}')
    
##### 600페이지 정도 수집하면 에러로 끝납니다. 추후 수정할 예정

In [5]:
len(url_list) # 수집된 게시물 수 

4000

##### 본문 스크랩

In [6]:
title_list = []
writer_list = []
date_list = []
text_list = []
tag_list = []

for i in tqdm(range(len(url_list))):
    driver.get(url_list[i])
    element = driver.find_element_by_id("mainFrame") #iframe 태그 엘리먼트 찾기
    driver.switch_to.frame(element) #프레임 이동
    
    title = driver.find_element_by_class_name('pcol1').text # 제목 
    title_list.append(title)
    writer = driver.find_element_by_class_name('nick').text # 글쓴이
    writer_list.append(writer)
    time.sleep(1)
    try:
        date = driver.find_element_by_class_name('se_publishDate.pcol2').text # 작성일 
        date_list.append(date)
    except:
        date_list.append("NULL")
    try:
        text = driver.find_element_by_class_name('se-main-container').text.replace('\n',' ') # 본문 
        text_list.append(text)
    except:
        text_list.append("NULL")
    
    try: 
        tags = driver.find_elements_by_class_name('item.pcol2.itemTagfont._setTop')
        tags_dummy = []
        for j in range(len(tags)):
            tag = driver.find_elements_by_class_name('item.pcol2.itemTagfont._setTop')[j].text.replace('#','')  # 해쉬태그
            tags_dummy.append(tag)
        tag_list.append(tags_dummy)
    except:
        tag_list.append("NULL")
        
    

100%|██████████| 4000/4000 [3:09:36<00:00,  2.84s/it]  


In [22]:
print(len(url_list))
print(len(title_list))
print(len(writer_list))
print(len(tag_list))
print(len(writer_list))
print(len(text_list))

4000
4000
4000
4000
4000
4000


##### DataFrame화

In [23]:
blog = pd.DataFrame(
    {
        "dataSource":"NAVER_BLOG",
        "searchKeyword":"부산 맛집",
        "postingUrl":url_list,
        "title":title_list,
        "author":[writer_list[x].replace('\n','') for x in range(len(writer_list))],
        "postingDate":date_list,
        "postingText":text_list,
        "hashtags":tag_list
    }
)

In [24]:
blog

,dataSource,searchKeyword,postingUrl,title,author,postingDate,postingText,hashtags
0,NAVER_BLOG,부산 맛집,https://blog.naver.com/bomypapa/222507501262,대박이었던 부산 맛집 소개,ebom,2021. 9. 16. 10:36,얼마 전 친구들과 유명한 부산 맛집이 롯데백화점에 입점했다고 해 곧장 찾아가 봤어요...,"[부산맛집, 모시모시롯데백화점광복점]"
1,NAVER_BLOG,부산 맛집,https://blog.naver.com/youriful/222553223837,백점이었던 부산 맛집 모음,유리풀,2021. 10. 30. 11:51,지난주 지인들과 모임이 있어 미리 찾아봐뒀던 부산 맛집에서 식사를 했어요. 신선한...,[부산맛집]
2,NAVER_BLOG,부산 맛집,https://blog.naver.com/mizuhara411/222542021435,인상깊었던 부산 맛집 공유,비비빅,2021. 10. 20. 8:10,얼마 전 지인들과 모임을 위해 미리 검색해둔 부산 맛집에 방문했습니다. 선도 높은 ...,"[부산맛집, 모시모시]"
3,NAVER_BLOG,부산 맛집,https://blog.naver.com/sosmarrin/222530392743,필수코스였던 부산 맛집 리스트,승현(sosmarrin),2021. 10. 8. 1:38,친구들과 주말에 떠났던 부산 여행에서 다들 같이 부산 맛집 투어를 다녀봤어요. 그중...,[]
4,NAVER_BLOG,부산 맛집,https://blog.naver.com/whgiddnjs/222317058957,호불호 없는 부산 맛집 리스트,미미쵸코(whgiddnjs),2021. 4. 20. 20:37,얼마 전 친구네 집들이를 갔다가 현지인들도 인정한다는 부산 맛집을 방문했어요. 다...,[]
...,...,...,...,...,...,...,...,...
3995,NAVER_BLOG,부산 맛집,https://blog.naver.com/ggll0000/222253419242,(부산맛집) 부산에 가면 꼭 가야하는 해운대가야밀면,새우새우(ggll0000),2021. 2. 23. 11:38,"안녕하세요. 오늘은 부산에 밀면,비빔밀면,만두로으로 유명한 곳이 있다고 해서 해운...","[부산맛집, 해운대맛집추천, 해운대밀면맛집, 해운대밀면, 밀면전문점, 해운대놀거리,..."
3996,NAVER_BLOG,부산 맛집,https://blog.naver.com/ddobuck/222281656191,부산 맛집 : 춘하추동밀면 서면본점,또벅또벅(ddobuck),2021. 3. 21. 12:50,"""춘하추동밀면 서면본점"" 부산 맛집 탐방 안녕하세요 또벅이입니다 😊 날씨가 한껏 풀...","[내돈내산, 춘하추동밀면, 서면맛집, 부산맛집, 부산밀면, 밀면맛집, 부산먹방]"
3997,NAVER_BLOG,부산 맛집,https://blog.naver.com/call1028/222405553532,송정삼대국밥 / 부산 맛집 / 서면 맛집,someone(call1028),2021. 6. 21. 20:25,갑자기 비오는 날 밥종류로 먹으려고 주변검색으로 메뉴 찾았는데 맛집이었다 ! 맛있는...,[]
3998,NAVER_BLOG,부산 맛집,https://blog.naver.com/tpwls7062/222410232607,"[부산맛집] 뚜벅이 부산여행으로 방문한 멘즈키, 오엘이커피",ssezi(tpwls7062),2021. 6. 25. 21:48,부산에 도착하자마자 방문한 곳은 “멘즈키”!! 츠케멘을 파는 가겐데 면을 육수에 적...,"[부산, 부산맛집, 존맛, 츠케멘, 라멘, 부산여행, 맛집추천, 오엘이커피, ole..."


##### csv file로 저장

In [26]:
path = '/home/haewon/바탕화면/'
file_name = 'Posting_NaverBlog_부산맛집-1124'

In [27]:
blog.to_csv(f'{path}{file_name}.csv')